In [38]:
!pip install web3
import json
from web3 import Web3
import pandas as pd
import numpy as np
import networkx as nx 
from tqdm.notebook import tqdm
import math

You should consider upgrading via the 'c:\users\fedem\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [39]:
# StackOverflow ♥
# we need to filter out all those paths that ping-pongs during the path 
def findPaths(G,u,n):
    if n==0:
        return [[u]]
    paths = [[u]+path for neighbor in G.neighbors(u) for path in findPaths(G,neighbor,n-1)]
    return paths

def find_cycles(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2]

def find_cycles_not_PingPong(G,u,n):
    paths = findPaths(G,u,n)
    return [tuple(path) for path in paths if (path[-1] == u) and sum(x ==u for x in path) == 2 and sum(prev==next for prev,cur,next in zip([None]+path[:-1], path, path[1:]+[None]))==0] #1) check that the last element is the starting element; 2) check that the starting element is counted just twice in the path; 3) check that previous element of the path is different from the subsequent one

In [40]:
def saveJson(path,file):
    with open(path, 'w') as f:
        json.dump(file, f)

In [52]:
with open(f"endpoints/baseTokens.json") as f:
    info_json = json.load(f)
_baseToken = info_json

chain = "Fantom"
dexes = ["SpiritSwap","SpookySwap","SushiFTM"]
_testingQuantity = 5.5151
_wEth = _baseToken[chain]
thresholdNumber = 5 #minimum number of pools (=edges)
tokensInvolvedInCycle = 3 #Put 3 for triangular abitrage
showGraphs = False
onlyDifferentDexes = True

_userAccount = "0xD59488d45304b2C7DC593f491F7206073602854B"

In [42]:
pools = []
for i in range(len(dexes)):
    with open(f"pools/Pools{dexes[i]}.json") as f:
        info_json = json.load(f)
    pools.append(list(info_json))

allTokens = np.unique(np.concatenate(list(map(lambda x:np.array(list(map(lambda y:y["tokens"],x))).flatten(),pools))).ravel())

poolMatrix = {}
for i in tqdm(range(0,len(allTokens))): #iterates over the i-th token
    obj = {}
    for j in range(0,len(pools)): #iterates over the j-th dex 
        for z in range(len(pools[j])): #iterates over the z-th pool of the j-th dex
            if(allTokens[i] in pools[j][z]["tokens"])==True:
                otherIndex = 1 if pools[j][z]["tokens"].index(allTokens[i])==0 else 0
                obj[pools[j][z]["tokens"][otherIndex]]=[pools[j][z]["indirizzoPool"],dexes[j]]
    poolMatrix[allTokens[i]]=obj
#saves data as 
#i-thToken{
#   otherToken:[poolAddress, j-thDex] → in this way the pool is overwritten tho
# }
# better saving as 
#i-thToken{
#   otherToken:[poolAddress, j-thDex] → in this way the pool is overwritten tho
# }


  0%|          | 0/14958 [00:00<?, ?it/s]

In [ ]:
saveJson("mock.json",poolMatrix)
print(poolMatrix)

In [43]:
dataframe = pd.DataFrame(poolMatrix)
dataframe = dataframe.fillna(0)
dataframe

,0x00000087C4ceBfFb95746d1935DE7fBcAb092F40,0x0003C4dCA25C68Ce7296Adb383019A775B7Fbb15,0x00056e6b50732D8418D4B4BFcE77084800eFbe14,0x00086E15E2de1b87accFa8CFFf705223d045b474,0x000E6B94fEdB952022bDDF63A3227221460AA5ed,0x000E6EB00234eaAcbe3FC97ee39d0a20f97fAa86,0x0012E6E5cB6f8f8a37F53aCF95aeEE76DeF6Da02,0x00142fD23861A5FD634d56Fc5FB04389526ec161,0x001479933C26FeB43A410B3C935df2a0B49C48BC,0x0023c81f5499958cE66B34933A20cD8C89Ca83E7,...,0xff9204F9824a7a8AABB0B0d0B273d9468281F1FA,0xff996a227D93b3cdfb06aFE950009777B386d0C4,0xffCa34B6aE306480718E86e2ec40536E41cB3479,0xffD29590B02d1C413D5DdF3c0DD5E5894dd87666,0xffD4f02B4d965d194e98293e6CEAB207De6c46df,0xffD5e417CFB52c6C65C92c564595dA1AB4D20c13,0xffEFb50FC7cDF62047e3a8Cdb43bEC2B2F9e85EF,0xffa5b3C502FEC8962D4ED1432Bb5C9eDA0f0ca10,0xffcC4a5c6C0dcf58896421CB77A1750BcCE261a5,0xffd560B1B9613D6a04e0740fcE93828C77C77724
0x152eE697f2E276fA89E96742e9bB9aB1F2E61bE3,"[0xDdff57C9Ed0565c69352ecE70d28aB2807Db670B, S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x468003B688943977e6130F4F68F23aad939a1040,"[0x9eB0424ec3B9F2C3543002f56782307830BAD887, S...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x04068DA6C83AFCFA0e13ba15A6696662335D5B75,0,"[0x9710704F0c1b85352C689Db8D9584A97a41df203, S...",0,0,"[0x3C0E720F81f6B6EFf3567F138bE662Ea5CE5637d, S...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x8D11eC38a3EB5E956B052f67Da8Bdc9bef8Abf3E,0,0,"[0xE1504AF4BeABe29eECbc6353F458f67F8445266B, S...",0,"[0xE6A00a3246e6573C3eda977574E25595E6C66876, S...",0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83,0,0,0,"[0x85fF84925eDF6C84D8f92ab55214fcf6bf2B5D23, S...","[0xD6cfc4dA1390c90A6B831e652CdA13cf0D2AfA2e, S...","[0x64Ba3214aBEFeF7C6E3693B4914301626633aEC9, S...","[0x9B0B976ce3147E4D450270fb184BAC0Fa736Fc7B, S...","[0xa77c4e295eCc396E37d967eB2b5285A3445fC909, S...","[0x3D1220E3bFa7E09d7399432F48C5501fE87C7f4E, S...","[0x1A5532913C4d0E491B1D58fD5a74c627304F0094, S...",...,"[0x962c96d1dC21fa33D9514c0F6B21c34455330e63, S...","[0x8f1E3c6Fc438b645696125C7489F1bEBfe2F358e, S...","[0xeF2944faE30DD07ddc48829e80a34DF4Ebf5102d, S...","[0x5a686B3d08C773dE147275954d67B3ABA0F99afC, S...","[0x3FaD53a12FD1ae2100e269b29DB42BEb5450F3E6, S...","[0xf36df55C47EE0b335F596E20B3C6809362401686, S...","[0x652A8d06b02A4F1e4eFa4ecd2701C304a2aa8eaf, S...","[0x2b02445424f95a931BfAFcD464d9b9A5082A331b, S...","[0x70ae7dD3DbC17e3f04aae31010Df23950615467E, S...","[0xACDDc32ab18bFECe8C924BF286052E0b36dCba4e, S..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xbcc5b54F7F97bE2bb0c024Cc5F69367aAefE1DF7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xD45f71D9439EBf21c23b7eB5A9BcA61115Cb56b6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0x56eBc660DbF8411555bC0Ff1cb67377A4e1C2257,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0xF2aC8bc3b282F16CA55c5C91b3Ab47F78E4B1d76,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
dataframe[_wEth]

0x152eE697f2E276fA89E96742e9bB9aB1F2E61bE3                                                    0
0x468003B688943977e6130F4F68F23aad939a1040    [0xAB3F048f9a169D969Ea9fb2Fdf74Bc48107Bd573, S...
0x04068DA6C83AFCFA0e13ba15A6696662335D5B75    [0xA48869049e36f8Bfe0Cc5cf655632626988c0140, S...
0x8D11eC38a3EB5E956B052f67Da8Bdc9bef8Abf3E    [0xd32F2EB49E91aA160946f3538564118388d6246a, S...
0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83                                                    0
                                                                    ...                        
0xbcc5b54F7F97bE2bb0c024Cc5F69367aAefE1DF7                                                    0
0xD45f71D9439EBf21c23b7eB5A9BcA61115Cb56b6                                                    0
0x56eBc660DbF8411555bC0Ff1cb67377A4e1C2257                                                    0
0xF2aC8bc3b282F16CA55c5C91b3Ab47F78E4B1d76                                                    0
0xC9f7e7820868F206EfaE3c3436aBB49FBf45fF

In [45]:
arrayMatrix = dataframe.values
h, w = arrayMatrix.shape
G = nx.DiGraph()
for i in tqdm(range(h)): 
 for j in range(w):
   if arrayMatrix[i][j]!=0: 
     G.add_edge(i,j) 
import matplotlib.pyplot as plt 
plt.figure(5,figsize=(30,30))
ftmIndex = list(dataframe.columns).index(_wEth) #we point out node associated to FTM
color_map = ['red' if node == ftmIndex else '#1f78b4' for node in G]

  0%|          | 0/14958 [00:00<?, ?it/s]

<Figure size 2160x2160 with 0 Axes>

In [ ]:
if(showGraphs):
  nx.draw( G, node_size =50, node_color = color_map) 
  plt.show()

In [46]:
#We reduce the matrix to those tokens which have at least two different pools, eliminating recursively rows and columns
#In this way we eliminate nodes on the boundary and nodes that have very few links (=pools) with other tokens
filteredDataframe = dataframe
previousShape = (0,0)
while(filteredDataframe.shape!=previousShape):
    print(filteredDataframe.shape,previousShape)
    previousShape = filteredDataframe.shape
    data = (filteredDataframe[(filteredDataframe != 0).sum(1) >= thresholdNumber])
    data = data[data.index]#pd.DataFrame(poolMatrixReduced).loc[pd.DataFrame(poolMatrixReduced).columns]
    filteredDataframe = data

print(filteredDataframe.shape)
# filteredDataframe.to_json(f"Pools{dex}Filtered.json")

(14958, 14958) (0, 0)
(215, 215) (14958, 14958)
(178, 178) (215, 215)
(176, 176) (178, 178)
(175, 175) (176, 176)
(175, 175)


In [47]:
filteredDataframe

,0x468003B688943977e6130F4F68F23aad939a1040,0x04068DA6C83AFCFA0e13ba15A6696662335D5B75,0x8D11eC38a3EB5E956B052f67Da8Bdc9bef8Abf3E,0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83,0xAd84341756Bf337f5a0164515b1f6F993D194E1f,0x049d68029688eAbF473097a2fC38ef61633A3C7A,0x82f0B8B456c1A451378467398982d4834b6829c1,0x841FAD6EAe12c286d1Fd18d1d525DFfA75C7EFFE,0x74b23882a30290451A17c44f4F05243b6b58C76d,0x5Cc61A78F164885776AA610fb0FE1257df78E59B,...,0x433e10fBcb263595d7deDa39a6Ed515cbA85364A,0xf61cCdE1D4bB76CeD1dAa9D4c429cCA83022B08B,0x2055f39e549fdd42516B637dA80Db7D6776A238c,0xbB29D2A58d880Af8AA5859e30470134dEAf84F2B,0x23cBC7C95a13071562af2C4Fb1Efa7a284d0543a,0x04A95442738A2004069E7E6726b9df8E02794Be3,0x1b9F2aae1E984d3E7D44b3Dc67dE5f37184B73B2,0x89D5e71E275B4bE094Df9551627BCF4E3b24cE22,0xD7F17e44Ee7d39966fdbD979cFdC9bdD9e2C8A9f,0xcfF748A0D8fddf5bE3FbC47F3a1C7630E1Cff2Cd
0x468003B688943977e6130F4F68F23aad939a1040,0,"[0x2276e28645a35DcA7Cce0958eBf6451a998Dd1dc, S...",0,"[0xAB3F048f9a169D969Ea9fb2Fdf74Bc48107Bd573, S...","[0x1858ede71495B67181C9902292AC7AC7f0c61999, S...","[0xCCD974e0450FE9D84CD10c2F75F13a48972bB449, S...","[0x63F548b530b42ba1d1D0Dc160848D389DF3690D4, S...","[0x18F60C11dE3bE28c653db5d647DA2101995e8253, S...","[0xeF4b24f6858CdDf952F81CB9b55635B2042005A0, S...","[0x6998315e68924a879b77E6366494EE9812897D8F, S...",...,0,0,0,"[0xe1a81130d132a7F68FD9a6256d3C70916BfD52d0, S...",0,0,0,0,0,0
0x04068DA6C83AFCFA0e13ba15A6696662335D5B75,"[0x2276e28645a35DcA7Cce0958eBf6451a998Dd1dc, S...",0,"[0xbd0eEC56dA621B6b31E3d06614f7853624e1c0af, S...","[0xA48869049e36f8Bfe0Cc5cf655632626988c0140, S...","[0x19Aea462D6d917a75b8971711D0AF44F76e129d7, S...","[0x8Be92F3d64E91D08AB1CC8A5c487dA3f1695b11e, S...","[0xFFdc0531288dc91C1F49Db03A90Ed84725E9eDa7, S...","[0xf8Cb2980120469d79958151daa45Eb937c6E1eD6, S...","[0x829843DD57863662c63c66A4c3996515504d1e0D, S...","[0xD687dD4b2dca486AB4AaFed87990ecEf4B28c5A1, S...",...,0,0,0,0,0,0,0,0,0,0
0x8D11eC38a3EB5E956B052f67Da8Bdc9bef8Abf3E,0,"[0xbd0eEC56dA621B6b31E3d06614f7853624e1c0af, S...",0,"[0xd32F2EB49E91aA160946f3538564118388d6246a, S...","[0x673C97c8040a615444D3B56f3F17075c30fA3f0a, S...","[0xD0154BE15563c100A90C9B4499f54E0F2f3098c6, S...","[0xab2D42c1109102F7DDfa455749d424221fDBeB9A, S...","[0x45414EE53417963A88C839ee614EFe6E9459f2B2, S...","[0x71C8BceEcE3dAf9E27741D2Cc1F03170f862555f, S...","[0x240f161d87523142Ea2d027CECD111be499b68ee, S...",...,0,0,"[0xEB8656dd0c6EADFE67AE73055FDE7969B02d9F9F, S...",0,"[0x322a282Ef6978780A40aD0879dA78F08CbEDF6ed, S...",0,0,0,0,0
0x21be370D5312f44cB42ce377BC9b8a0cEF1A4C83,"[0xAB3F048f9a169D969Ea9fb2Fdf74Bc48107Bd573, S...","[0xA48869049e36f8Bfe0Cc5cf655632626988c0140, S...","[0xd32F2EB49E91aA160946f3538564118388d6246a, S...",0,"[0x8623836F527350Ec50691479674Df0cD7773810c, S...","[0xd019dd7C760c6431797d6ed170bFFb8FAee11F99, S...","[0x2ed684f14f951b467f2066c663F488cd0F0475E7, S...","[0x3CBe1476d86ccE439bB30201A389B690C8cC40aF, S...","[0x3d0BD54c48C2C433ea6fed609Cc3d5Fb7A77622B, S...","[0xe22d1953a21B6c93c4d129A0697a78aA87BD9831, S...",...,"[0xe0C4D423947cFEB817469744A3087DC08d8DB5d1, S...","[0x2f64270715BF7C33Af1aE0842c29bd75F1882102, S...","[0xb825e6a1C3105e4fEae1e128eE88615695Bb2518, S...","[0xAffA895bdf384E20A2C475d7681dFda4bAD7959C, S...","[0x987B81404828621A37d40ADD6e9dF99F84E9Fb33, S...","[0x35150583cABa766a24C2100111f238F525135290, S...","[0x03F736D3d23Cf277b053e9fd509024D866880e3c, S...","[0x1Ce91DB29efFAe74158a2Fe98eEEd500542031b7, S...","[0x5569d3F44727313cE383132b8F7578bd4947FB53, S...","[0x029C97F8019f01136536176bffFA60686E8478d8, S..."
0xAd84341756Bf337f5a0164515b1f6F993D194E1f,"[0x1858ede71495B67181C9902292AC7AC7f0c61999, S...","[0x19Aea462D6d917a75b8971711D0AF44F76e129d7, S...","[0x673C97c8040a615444D3B56f3F17075c30fA3f0a, S...","[0x8623836F527350Ec50691479674Df0cD7773810c, S...",0,"[0x8A6eBa896d750D8C04E815B74B3744d97BD0DAaD, S...",0,"[0xA1Dc91d30AeF8eCb1ECcABaFDEA48a3B9155b597, S...","[0x1d6101825d3412925e56C058559E6BFCC4661f1c, S...","[0x74772FBe8ea1b9aD1D750D

In [48]:
arrayMatrix = filteredDataframe.values
h, w = arrayMatrix.shape
G = nx.DiGraph()
for i in tqdm(range(h)): 
 for j in range(w):
  #  if arrayMatrix[i][j] !=0 and arrayMatrix[i][j] !="0x00": 
#    if "0x" in str(arrayMatrix[i][j]): 
   if arrayMatrix[i][j]!=0: 
      G.add_edge(i,j) 
import matplotlib.pyplot as plt 
plt.figure(5,figsize=(5,5))
ftmIndex = list(filteredDataframe.columns).index(_wEth) #we point out node associated to FTM
color_map = ['red' if node == ftmIndex else '#1f78b4' for node in G]

  0%|          | 0/175 [00:00<?, ?it/s]

<Figure size 360x360 with 0 Axes>

In [ ]:
if(showGraphs):
   nx.draw( G,node_size=50, node_color=color_map ) 
   plt.show()

In [49]:
#We define all the cycles that are present in such a matrix: we don't need to do it everytime, it's enough doing it just one time and save the path involving FTM as first input token
#Once we collected the paths, we will ask the information of the pools involved and we will apply recursively (for each path found) the "theorem" of the paper regarding existence of exploitable arbitrages   
# https://42papers.co/pdf/2105.02784.pdf

ftmIndex = list(filteredDataframe.columns).index(_wEth) #for future work, getting confidence with flashloans, starting node may be any "flashloanable" token, not only FTM
cycles = find_cycles_not_PingPong(G,ftmIndex,tokensInvolvedInCycle) #we extract every cycle (=ordered tuple of nodes) starting from the InputToken and with a length equal to "tokensInvolvedInCycle"

tokenPaths = []
poolsPaths = []
dexesPaths = [] 

#For each cycle, we translate the ordered tuple of nodes in a ordered tuple of tokens and in a ordered tuple of pools (with length n-1) 
for j in tqdm(range(0,len(cycles))):
    currentCycle = cycles[j]
    cycleTokens=[]
    poolsInvolved = []
    dexesInvolved = []
    for i in range(0,len(currentCycle)):
        cycleTokens.append(filteredDataframe.columns[currentCycle[i]])
        if(i>0):
            poolsInvolved.append((filteredDataframe[filteredDataframe.columns[currentCycle[i-1]]][filteredDataframe.columns[currentCycle[i]]])[0])
            dexesInvolved.append((filteredDataframe[filteredDataframe.columns[currentCycle[i-1]]][filteredDataframe.columns[currentCycle[i]]])[1])
    tokenPaths.append(cycleTokens)
    poolsPaths.append(poolsInvolved)
    dexesPaths.append(dexesInvolved)

  0%|          | 0/2028 [00:00<?, ?it/s]

In [53]:
# Now we remove all those paths in which a same pool is repeated several times in the same path
#Moreover, we check if there is any issue in the arbitrage path by simulating a multiswap, in order to discard all those paths involving scam tokens

with open("abis/Lifeguard.json") as f:
    info_json = json.load(f)
abi = info_json["abi"]

with open("endpoints/deployedAddresses.json") as f:
    info_json = json.load(f)
contract = info_json

with open("endpoints/indirizziDex.json") as f:
    info_json = json.load(f)
dexAddresses = info_json

with open("endpoints/chainsEndpoint.json") as f:
    info_json = json.load(f)
chainsEndpoint = info_json

url = chainsEndpoint[chain] #we don't consider multi-chain swaps
# url = "http://localhost:7545"
_routers = [dexAddresses[dexes[i]]["router"] for i in range(len(dexes))]
indirizzoContrattoCustom = contract[chain]#Mainnet 
testingQuantity = math.floor(_testingQuantity*(math.pow(10,18)))

w3 = Web3(Web3.HTTPProvider(url))
contrattoLifeguard = w3.eth.contract(address=indirizzoContrattoCustom, abi=abi)

#START FROM HERE!

finalPoolsPaths = []  # we will apply the "theorem" iterating on this list
# finalRoutersPaths = []
finalRoutersPaths2 = []
finalTokenPaths = []
for i in tqdm(range(0, len(poolsPaths))):
    if(len(set(poolsPaths[i])) == len(poolsPaths[i])):
        goodPath = False
        routerPath = [dexAddresses[dexesPaths[i][j]]["router"] for j in range(len(dexesPaths[i]))]
        if(onlyDifferentDexes and len(set(routerPath))==len(routerPath)):
            continue
        try:
            #I need to write first the function for multiSwap on different dexes
            #it must take as input the token path (dimension "n") and the path of routers (dimension "n/2"==dimension of poolPath) 
            # in the for cycle, it invokes the approve function of the i-th router address

            # DA FARE: 
            # goodPath = True 
            # goodPath = contrattoLifeguard.functions.multipleSwapUniCelo(tokenPaths[i],_router,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 }) if chain=="Celo" else contrattoLifeguard.functions.multipleSwapUni(tokenPaths[i],_router,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
            # goodPath = contrattoLifeguard.functions.multipleSwapMultiUni(tokenPaths[i],routerPath,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
            goodPath = contrattoLifeguard.functions.multipleSwapOnlyPools(tokenPaths[i],poolsPaths[i],False,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 }) if chain=="Celo" else contrattoLifeguard.functions.multipleSwapOnlyPools(tokenPaths[i],poolsPaths[i],True,True).call({ "from": _userAccount, "value": testingQuantity, "gas":5000000 })
        
        except Exception as e: print(e)
        
        if(goodPath):
            finalPoolsPaths.append(poolsPaths[i])
            finalTokenPaths.append(tokenPaths[i])
            # finalRoutersPaths.append(dexesPaths[i])
            finalRoutersPaths2.append(routerPath)
print(len(finalTokenPaths))
with open(f'monitorUtils/finalPoolsPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalPoolsPaths, f)
with open(f'monitorUtils/finalTokenPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalTokenPaths, f)
# with open(f'monitorUtils/finalDexesPaths{"_".join(dexes)}.json', 'w') as f:
#     json.dump(finalRoutersPaths, f)
with open(f'monitorUtils/finalDexesPaths{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalRoutersPaths2, f)
# finalTokenPaths

  0%|          | 0/2028 [00:00<?, ?it/s]

execution reverted: UniswapV2: INSUFFICIENT_OUTPUT_AMOUNT
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: INSUFFICIENT_OUTPUT_AMOUNT
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: TRANSFER_FAILED
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2Library: INSUFFICIENT_LIQUIDITY
execution reverted: UniswapV2: K
execution reverted: UniswapV2: K
execution reverted: UniswapV2: INSUFFICIENT_OUTPUT_AMOUNT
execution reverted: UniswapV2: INSUFFICIENT_OUTPUT_AMOUNT
execu

In [54]:
# finalPools1=filteredDataframe.to_numpy().flatten() 
finalPools1 = np.array(finalPoolsPaths).flatten() #pool addresses we will query on the smart contract (associated to tokens that have links at least with "thresholdNumber" other tokens)
finalPools = np.unique(finalPools1[finalPools1!=0])
with open(f'monitorUtils/snipedPools{"_".join(dexes)}.json', 'w') as f:
    json.dump(finalPools.tolist(), f)
print(len(finalPools))

949


C:\Users\Fedem\AppData\Local\Temp/ipykernel_27076/4025130058.py:3: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  finalPools = np.unique(finalPools1[finalPools1!=0])
